# Composite Beam Example

## Setup

Add directories containing the st7py package and the St7API module (shipped with Strand7 and also included in lib/).


In [1]:
import sys
sys.path.append("../.")
sys.path.append("../lib")


## Initialize API and interact with the model

In [2]:
import st7py as st7

st7.start()

model = st7.Model(filename='../models/beam_composite.st7', 
                  scratch='C:\Temp', 
                  uid=1)

model.open()

tots = model.totals(disp=True)

St7API initialized
Element totals:
		Nodes: 1333
		Beams: 132
		Plates: 1056
		Bricks: 0


## Run NFA Solver


In [3]:
nfa = st7.NFA(uid = 1,
             filename = '../models/beam_composite.NFA',
             logname = '../models/beam_composite.NFL',
             fcase = 1,
             nsm = (1,),
             nmodes = 4)

nfa.run()

freqs = nfa.getFrequencies(disp=True)

NFA solver completed successfully (uid: 1)
Mode 1    Frequency: 4.688264508471357 Hz
Mode 2    Frequency: 7.201602906205432 Hz
Mode 3    Frequency: 16.67565035865051 Hz
Mode 4    Frequency: 21.257994329482404 Hz


## Close Model and Unload API

In [4]:
model.close()
st7.stop()

St7API released
